In [1]:
import os
import gzip
import shutil
import tensorflow as tf
import six.moves.urllib_request as urllib
tf.enable_eager_execution()
from sklearn.model_selection import train_test_split



/Users/liuxinxin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# data url
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz"

In [3]:
# google download code
# if downloaded , done else download

def download_report_hook(count, block_size, total_size):
  """Report hook for download progress.

  Args:
    count: current block number
    block_size: block size
    total_size: total size
  """
  percent = int(count * block_size * 100 / total_size)
  print("\r%d%%" % percent + " completed", end="\r")


def maybe_download(directory, filename, uri):
  """Download filename from uri unless it's already in directory.

  Copies a remote file to local if that local file does not already exist.  If
  the local file pre-exists this function call, it does not check that the local
  file is a copy of the remote.

  Remote filenames can be filepaths, any URI readable by tensorflow.gfile, or a
  URL.

  Args:
    directory: path to the directory that will be used.
    filename: name of the file to download to (do nothing if it already exists).
    uri: URI to copy (or download) from.

  Returns:
    The path to the downloaded file.
  """
  tf.gfile.MakeDirs(directory)
  filepath = os.path.join(directory, filename)
  if tf.gfile.Exists(filepath):
    tf.logging.info("Not downloading, file already found: %s" % filepath)
    return filepath

  tf.logging.info("Downloading %s to %s" % (uri, filepath))
  try:
    tf.gfile.Copy(uri, filepath)
  except tf.errors.UnimplementedError:
    if uri.startswith("http"):
      inprogress_filepath = filepath + ".incomplete"
      inprogress_filepath, _ = urllib.urlretrieve(
          uri, inprogress_filepath, reporthook=download_report_hook)
      # Print newline to clear the carriage return from the download progress
      print()
      tf.gfile.Rename(inprogress_filepath, filepath)
    else:
      raise ValueError("Unrecognized URI: " + filepath)
  statinfo = os.stat(filepath)
  tf.logging.info("Successfully downloaded %s, %s bytes." %
                  (filename, statinfo.st_size))
  return filepath

In [4]:
# download the file to the data documents
# now you can get the covtype.data.gz file in your document
file_name = url.split('/')[-1]
maybe_download("",file_name, url)

INFO:tensorflow:Not downloading, file already found: covtype.data.gz


'covtype.data.gz'

In [5]:
# extract the data
# now you get covtype.data file
new_file_name = file_name.replace(".gz", "")
with gzip.open(file_name, 'rb') as f_in:
    with open(new_file_name, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [6]:
# devide train/test
# you get covtype.csv.train and covtype.csv.test file
data = []
with open(new_file_name,'r') as f:
    data = f.readlines()
train,test= train_test_split(data,test_size=0.1)

def save_file(name,dataset):
    with open(name,'w') as f:
        for data in dataset:
            f.write(data)

save_file('covtype.csv.train',train)
save_file('covtype.csv.test',test)

In [7]:
# we can forward to the tutorail
# maybe google can print more beatiful
defaults = [tf.int32] * 55
dataset = tf.contrib.data.CsvDataset(['covtype.csv.train'],defaults)
print(list(dataset.take(1)))

Instructions for updating:
Use `tf.data.experimental.CsvDataset(...)`.
[(<tf.Tensor: id=68, shape=(), dtype=int32, numpy=2679>, <tf.Tensor: id=69, shape=(), dtype=int32, numpy=32>, <tf.Tensor: id=70, shape=(), dtype=int32, numpy=9>, <tf.Tensor: id=71, shape=(), dtype=int32, numpy=175>, <tf.Tensor: id=72, shape=(), dtype=int32, numpy=8>, <tf.Tensor: id=73, shape=(), dtype=int32, numpy=1347>, <tf.Tensor: id=74, shape=(), dtype=int32, numpy=218>, <tf.Tensor: id=75, shape=(), dtype=int32, numpy=221>, <tf.Tensor: id=76, shape=(), dtype=int32, numpy=139>, <tf.Tensor: id=77, shape=(), dtype=int32, numpy=1403>, <tf.Tensor: id=78, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=79, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=80, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=81, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=82, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=83, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=84, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=85, shape=

In [8]:
# col names 
# come from https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.info spend most of my time :/
col_names = [
    'Elevation',
    'Aspect',
    'Slope',
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am',
    'Hillshade_Noon',
    'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points',
    'Wilderness_Area',
    'Soil_Type',
    'Cover_Type'
]

In [9]:
# parsing every line
def _parse_csv_row(*vals):
    soil_type_t = tf.convert_to_tensor(vals[14:54])
    feat_vals = vals[:10] + (soil_type_t,vals[54])
    features = dict(zip(col_names,feat_vals))
    
    class_label = tf.argmax(vals[10:14],axis=0)
    return features,class_label

In [10]:
# apply the parse to the dataset
# Until here is part 1
dataset = dataset.map(_parse_csv_row).batch(64)
print(list(dataset.take(1)))

[({'Elevation': <tf.Tensor: id=313, shape=(64,), dtype=int32, numpy=
array([2679, 3048, 2846, 3156, 3299, 3261, 3179, 3036, 2725, 3210, 3144,
       3205, 2351, 2180, 3233, 2492, 2818, 2748, 3273, 2922, 2870, 3061,
       2838, 3211, 3034, 3275, 3018, 3252, 3065, 3164, 3021, 2965, 2720,
       2848, 2499, 2940, 2934, 3085, 1962, 2962, 2851, 3032, 3146, 2549,
       2894, 2360, 3108, 3214, 3130, 2923, 3005, 3065, 2626, 3049, 3109,
       3014, 2852, 2808, 3160, 2824, 3167, 2600, 3237, 3364], dtype=int32)>, 'Aspect': <tf.Tensor: id=312, shape=(64,), dtype=int32, numpy=
array([ 32, 122,  28,  96, 339, 291,  84, 219,  67, 293, 142,  85,  39,
        84,  94, 231, 139,  50, 272, 265, 144, 310,   3, 117, 198, 123,
        20, 228, 234,  88,  96,  86, 101, 307,  43, 352, 196,  48, 121,
       359, 124,  25,  82,  19,  16,  34, 333, 321, 163,  69, 110, 208,
        13, 344, 285, 345,  59,  86, 253, 131, 302,  17,  58,  84],
      dtype=int32)>, 'Slope': <tf.Tensor: id=320, shape=(64,), dtype=i